In [ ]:
using DrWatson
@quickactivate "dare"

using ReinforcementLearning
using IntervalSets
using LinearAlgebra
using ControlSystems
using CUDA
using DataFrames
using Plots

include(srcdir("nodeconstructor.jl"))

include(srcdir("env.jl"));

In [ ]:
Base.@kwdef mutable struct PV_module
    I_0 = 2.0381e-10           # Diode seturation
    ni = 1.2                   # Diode ideality factor for Si-mono
    k = 1.3806e-23             # Boltzman constant
    q = 1.6022e-19             # Electron charge
    T_0 = 273                  # Temperature offset
    N_cell = 36                 # Number of cells
#     V_T = k*T/q                # thermal voltage
    
#     R_s = 0.45                 # Serial resistance
#     R_p = 310.0248             # Parallel resistance
    G_ref = 1000               # Irradiance at STC
    mu_sc = 1.3e-3             # Coefficient tempreture of short circuit current
    I_ph_ref = 3.11            # Phototcurrent at STC
    
end;

In [ ]:
Base.@kwdef mutable struct PV_array
    pv_module::PV_module           # module parameters
    serial = 10
    parallel = 4
    connection = 's'
    
end;

In [ ]:
Test_PV = PV_module();

In [ ]:
function get_I(self::PV_module, V, G, T)
    function I_photo(self::PV_module, G, T)
        dT = self.T_0 + T
        I_ph = G/self.G_ref*(self.I_ph_ref + self.mu_sc * dT)
        return I_ph
    end;

    function I_diode(self::PV_module, V, G, T)
        dT = self.T_0 + T
        V_T = self.k*dT/self.q
        I_d = self.I_0*(exp(V/(self.ni*self.N_cell*V_T))-1)
        return I_d
    end;
    
    I = I_photo(self, G, T) - I_diode(self, V, G, T)
    return I
end

In [ ]:
function get_V(self::PV_module, I, G, T)
    function I_photo(self::PV_module, G, T)
        dT = self.T_0 + T
        I_ph = G/self.G_ref*(self.I_ph_ref + self.mu_sc * dT)
        return I_ph
    end;
    
    I = maximum([0,I])
    res = (I_photo(self, G, T)-I)
    
    dT = self.T_0 + T
    V_T = self.k*dT/self.q
    
    if res <= 0
        V=0
    else
        V = self.ni*self.N_cell*V_T*(log((res)/self.I_0)+1)
    end
    
    return V
end

In [ ]:
function get_I_test(pv_arr::PV_array, V, G, T)

    self = pv_arr.pv_module
    function I_photo(self::PV_module, G, T)
        dT = self.T_0 + T
        I_ph = G/self.G_ref*(self.I_ph_ref + self.mu_sc * dT)
        return I_ph
    end;

    function I_diode(self::PV_module, V, G, T)
        dT = self.T_0 + T
        V_T = self.k*dT/self.q
        I_d = self.I_0*(exp(V/(self.ni*self.N_cell*pv_arr.serial*V_T))-1)
        return I_d
    end;
    
    I = (I_photo(self, G, T) - I_diode(self, V, G, T)) * pv_arr.parallel
    return I
end;

function get_V_test(pv_arr::PV_array, I, G, T)

    self = pv_arr.pv_module
    function I_photo(self::PV_module, G, T)
        dT = self.T_0 + T
        I_ph = G/self.G_ref*(self.I_ph_ref + self.mu_sc * dT)
        return I_ph
    end;
    
    I = maximum([0.0,I[1]])
    res = I_photo(self, G, T)-(I/pv_arr.parallel)
    
    dT = self.T_0 + T
    V_T = self.k*dT/self.q
    
    if res <= 0
        V=0
    else
        V = self.ni*self.N_cell*pv_arr.serial*V_T*(log((res)/self.I_0)+1)
    end
    
    return V
end;

pv_module = PV_module()

pv_array = PV_array(;pv_module);

In [ ]:
v_test = 150

In [ ]:
i_test = get_I_test(pv_array, v_test, 1000, 27)

In [ ]:
i_test2 = 13.999444539034302

In [ ]:
v_test2 = get_V_test(pv_array, i_test2, 1000, 27)

In [ ]:
# i_d                        # Diode current
# v_d                        # Diode voltage
# i_0                        # Diode seturation
ni = 1.2                   # Diode ideality factor for Si-mono
k = 1.3806e-23             # Boltzman constant
q = 1.6022e-19             # Electron charge
T = 20                     # Cell temperature
N_cell = 36                # Number of cells
V_T = k*T/q                # thermal voltage

R_s = 0.45                 # 
R_p = 310.0248             # 
G_ref = 1000               # Irradiance at STC
mu_sc = 1.3e-3             # Coefficient tempreture of short circuit current
I_ph_ref = 3.11            # Phototcurrent at STC
I_0 = 2.0381e-10;

In [ ]:
serial = 10

In [ ]:
parallel = 4

In [ ]:
G = collect(200:200:1000)

In [ ]:
T = 25

In [ ]:
V = 0:1:500

In [ ]:
dT = 273 + T
V_T = k*dT/q   
I_ph = G./G_ref*(I_ph_ref + mu_sc * dT).* ones(length(V))';

In [ ]:
I_d = I_0*(exp.(V/(ni*N_cell*serial*V_T)).-1).*ones(5)';

In [ ]:
I = (I_ph' - I_d) * parallel;

In [ ]:
V_ = V.*ones(5)';

In [ ]:
I_nn = I[I .>= zero(I)];

In [ ]:
P = I .* V_;

In [ ]:
labels = ["200 W/m^2" "400 W/m^2" "600 W/m^2" "800 W/m^2" "1000 W/m^2"];

In [ ]:
plot(V_, I, xlim=(0,300),ylim=(0,20),title="PV modules - I(V)", label=labels, xlabel="V", ylabel="I")
scatter!([v_test], [i_test], color="blue", markershape=:star5, label="Test")

In [ ]:
plot(V_, P, xlim=(0,1), ylim=(0,5),title="PV modules - P(V)", label=labels, xlabel="V", ylabel="P")

In [ ]:
plot(I, V_, xlim=(0,30), ylim=(0,300),title="PV modules - V(I)", label=labels, xlabel="I", ylabel="V")
scatter!([i_test2],[v_test2], color="blue", markershape=:star5, label="Test")

In [ ]:
[i_test2]

In [ ]:
[v_test2]

In [ ]:
plot(I, P, xlim=(0,5), ylim=(0,50),title="PV modules - V(I)", label=labels, xlabel="I", ylabel="P")

## PV module V(I)

In [ ]:
I_ph = G[5]./G_ref*(I_ph_ref + mu_sc * dT)
I = collect(0:0.001:3.497)
V = V_T*log.((I_ph.-I)/I_0)

In [ ]:
plot(I, V, xlim=(0,5), ylim=(0,1),title="PV modules - V(I)", label=labels, xlabel="I", ylabel="V")

## With R_s and R_p

In [ ]:
dT = 273 + T
V_T = k*dT/q   
I_ph = G./G_ref*(I_ph_ref + mu_sc * dT)*ones(length(V))';
I_d = I_0*(exp.(V/(ni*N_cell*V_T)).-1).*ones(5)';

I_init = 1 .*ones(5)

R_s = 0.45                 # 
R_p = 310.0248             # 
        

function get_I(G, V)
    
    I_ph = ones(length(V)) * G/G_ref*(I_ph_ref + mu_sc * dT);
    delta = 0.00000000001
    
    function I_d(V, I)        
        a = ni*N_cell*V_T
        R_s = 0.45 
        return I_0*(exp.(V.+R_s.*I/(a)).-1)
    end

    function I_p(V, I)
        R_p = 310.0248
        R_s = 0.45 
        return V.+R_s.*I/R_p
    end
    
    function f_dot(V,I)
        a = ni*N_cell*V_T
        R_p = 310.0248
        R_s = 0.45 
        return -I_0*1/a*exp.(V.+R_s.*I/(a)).-1/R_p
    end
    
    I = 3*ones(length(V))
    I_= zeros(length(V))
    
    while true
        
        I_ = I - (I_ph - I_d(V,I) - I_p(V,I))./f_dot(V,I)
        
        check = sum(broadcast(abs, (I_-I)) .< delta)
        if check == length(V)
            break
        end
        I = I_
    end
            
    return I_
end
    

In [ ]:
V = 0:0.01:5

In [ ]:
I =get_I(1000, V)

In [ ]:
plot(V, I,ylim=(0,5),title="PV modules - I(V)", xlabel="V", ylabel="I")

In [ ]:
V = 0:0.01:1

In [ ]:
a = ni*N_cell*V_T
R_s = 0.45
R_p = 310.0248

In [ ]:
-I_0*1/a*exp.(V.+R_s.*I/(a)).-1/R_p

In [ ]:
I_0*(exp.(V.+R_s.*I/(a)).-1)

In [ ]:
sum(broadcast(abs, (I_-I)) .> delta)

In [ ]:
x=4

while true
    x_ = x - 3x^3/9x^2
    
    if abs(x_ - x) < 0.00001
        
        println(x_)
        break
    end
    x = x_
end